Fruit Inspection Quality:

Below is a Convolutinal Neural Netowrk(CNN) to classify fruits into fresh and rotten.

In [9]:
import tensorflow as tf  # A famous library used to create image classifiers amongst other things
# Tensorflow keras provides a user-friendly interface to build Deep Neural Networks - A Neural Network with multiple layers
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Check if TensorFlow is using GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Define paths
train_dir = 'C:\\Users\\Admin\\Desktop\\GAIP Proj Dataset\\Train'
test_dir = 'C:\\Users\\Admin\\Desktop\\GAIP Proj Dataset\\Test'

# Image Data Generator for augmentation
'''Augmentation is done to account for real-world variation (in size, orientation, zoom, etc.),
reducing overfitting, generating wider variations'''
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,  # randomly rotate between 0 and 40 degrees
    width_shift_range=0.2,  # randomly increases or decreases image width by 20%
    height_shift_range=0.2,  # randomly increases or decreases image height by 20%
    shear_range=0.2,  # randomly shearing (slanting) images towards the left or right up to 20%
    zoom_range=0.2,  # zooming in/out up to 20%
    horizontal_flip=True,  # horizontal flip
    fill_mode='nearest',  # fills empty pixels with the nearest color
)

# Only rescaling done for test set
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Generators
'''train_datagen applies the transformations defined above to the directory 
   loaded from train_dir using flow_from_directory function
   Images are resized to 150x150 pixels
   Batches of 32 preprocessed (augmented) images are passed at once to the neural network
   This is done to increase memory efficiency and speed
   We have 2 labels - fresh and rotten'''
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

'''test_datagen applies the transformations defined above to the directory 
   loaded from test_dir using flow_from_directory function
   Images are resized to 150x150 pixels
   Batches of 32 preprocessed (augmented) images are passed at once to the neural network
   This is done to increase memory efficiency and speed
   We have 2 labels - fresh and rotten'''
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# CNN model
'''
    1. We initialize a sequential model meaning the neural network's layers are stacked on top of each other.
    2. We add the first convolutional layer - a layer that takes an image input
       and processes the images to find underlying patterns in order to differentiate it from other classes of images.
       Conv2D(32, (3, 3)): Adds a convolutional layer with 32 filters, each of size 3x3.
       activation='relu': Uses the ReLU activation function, which helps the model learn complex patterns 
       by introducing non-linearity.
       A filter/kernel is a 3D matrix (of weights) that slides over the image to scan it to extract features of the image. 
       input_shape=(150, 150, 3): The input images have a size of 150x150 pixels with 3 color channels (RGB).
    3. A feature map (another 3D matrix) is the result of applying filter to an input.
    4. A max pooling layer reduces the spatial dimensions of a feature Map converting the 3D feature map to say 2D matrix or 
       an array for faster computation.
    5. We repeat steps 2 to 4 thrice again for higher level feature extraction - to obtain abstract level features thereby
       refining our understanding. Each layer builds on top of another. Number of filters used is different in each case.
    6. Flattens the 3D output of the previous layers to 1D vectors so it can be fed to the fully connected (FC) layers.
    7. First Dense - FC layer - Adds a fully connected layer with 512 neurons and uses the ReLU activation function.
    8. Adds a dropout layer that randomly sets 50% of the input neurons to zero during each training step. 
       This helps prevent overfitting by making the network more robust and ensuring it doesn't rely too heavily
       on any one feature.
    9. Adds a final dense layer with 1 neuron and uses the sigmoid activation function.
       This layer outputs a single value between 0 and 1, which is ideal for binary classification tasks 
       (like classifying between fresh and rotten fruit).
       
       
    So in summary the convolutional and max pooling layers extract features from images while FC layers classify it. 
      
'''
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),  #Adds a convolutional layer with 32 filters, each of size 3x3
    MaxPooling2D(2, 2),  # Reduces the spatial dimensions of the feature map
    Conv2D(64, (3, 3), activation='relu'),  # Adds another convolutional layer with 64 filters, each of size 3x3
    MaxPooling2D(2, 2),  # Reduces the spatial dimensions of the feature map
    Conv2D(128, (3, 3), activation='relu'),  # Adds another convolutional layer with 128 filters, each of size 3x3
    MaxPooling2D(2, 2),  # Reduces the spatial dimensions of the feature map
    Conv2D(128, (3, 3), activation='relu'),  # Adds another convolutional layer with 128 filters, each of size 3x3
    MaxPooling2D(2, 2),  # Reduces the spatial dimensions of the feature map
    Flatten(),  # Flattens the 3D output to 1D vector
    Dense(512, activation='relu'),  # Adds a fully connected layer with 512 neurons using ReLU activation function
    Dropout(0.5),  # Adds dropout to prevent overfitting
    Dense(1, activation='sigmoid')  # Adds the final output layer with sigmoid activation for binary classification
])

'''1. Loss function is a measure of how wrong the model's predictions are compared to the actual outputs (labels).
      'binary_crossentropy' is used when the model is predicting between two choices (like yes/no or 0/1).
   2. Optimizer is how the model updates itself based on the data it sees and the loss function.
      Adam is just a fancy name for a specific way of doing this, and 'learning_rate=1e-4' 
      means the model will learn slowly at first, which can sometimes help it learn better.
   3. 'accuracy' is a common way to measure how often the model predicts correctly.
'''
from tensorflow.keras.optimizers import Adam

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])

# Train the model
# Epoch refers to the number of cycles implemented during training.
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test accuracy: {test_acc * 100:.2f}%")

# Function to predict image class
def predict_image_class(image_path):
    '''Using data augmentation to make an image classifier work on the data you want to
    classify'''
    img = keras_image.load_img(image_path, target_size=(150, 150))
    x = keras_image.img_to_array(img)
    x = x / 255.0  # Rescale to [0, 1]
    x = x.reshape((1,) + x.shape)
    prediction = model.predict(x)
    if prediction < 0.5:
        return 'Fresh'
    else:
        return 'Rotten'

# Example usage of predict_image_class function
image_path = '"C:\Users\Admin\Desktop\GAIP Proj Dataset\Train\Rotten\1 (2).jpg"'
predicted_class = predict_image_class(image_path)
print(f"Predicted class for {image_path}: {predicted_class}")

Num GPUs Available:  0
Found 6253 images belonging to 2 classes.
Found 1604 images belonging to 2 classes.
Epoch 1/30
195/195 ━━━━━━━━━━━━━━━━━━━━ 273s 1s/step - accuracy: 0.5257 - loss: 0.6903
Epoch 2/30
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 81us/step - accuracy: 0.5000 - loss: 0.7243   
Epoch 3/30
195/195 ━━━━━━━━━━━━━━━━━━━━ 266s 1s/step - accuracy: 0.6321 - loss: 0.6360
Epoch 4/30
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 132us/step - accuracy: 0.7500 - loss: 0.5901  
Epoch 5/30
195/195 ━━━━━━━━━━━━━━━━━━━━ 275s 1s/step - accuracy: 0.7417 - loss: 0.5293
Epoch 6/30
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 89us/step - accuracy: 0.7188 - loss: 0.5423   
Epoch 7/30
195/195 ━━━━━━━━━━━━━━━━━━━━ 308s 2s/step - accuracy: 0.8291 - loss: 0.4010
Epoch 8/30
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 115us/step - accuracy: 0.8750 - loss: 0.4139  
Epoch 9/30
195/195 ━━━━━━━━━━━━━━━━━━━━ 262s 1s/step - accuracy: 0.8729 - loss: 0.3334
Epoch 10/30
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 94us/step - accuracy: 0.8125 - loss: 0.4638   
Epoch 1

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Admin\\Desktop\\GAIP Proj Dataset\\Train\\Rotten'

In [10]:
# Save the trained model
model.save('/content/drive/MyDrive/GAIP Proj Dataset/fruit_freshness_model.h5')
